Guide My Sleigh Data Overview:

The crew here at Data Fusion want to help ensure that the the joy and cheer of SantaCon comes with great bars, no lines and a map of fun and getting you back to your hotel with no extra thougts about logistics. Santas have done enough work all year long. 

Below we will be extracting data from the santacon.nyc website of the availible bars. We will also be listing 3 hotels that are also offering reduced night stay rates for our Santa's so they don't have to worry about lodging. 

After gathering this data, we will go ahead and make a few "Sleigh Rides" that will be a currated group of bars that Santa's will have already paid covers for. Allowing them no wait access to those bars. 

An algorithm will calculate the optimal commute (shorest walking distance and order)for that Sleigh Ride and getting our Santas back to their hotel. That information will be passed onto our users visually in the "Guide My Sleigh" app.

In [1]:
import json
import pandas as pd
from datetime import time
import openrouteservice
from itertools import permutations


## Bar Data

In [2]:
with open("santacon_2024_venues.json", "r") as file:
    json_data = json.load(file)

In [3]:
# Convert the nested JSON object (assuming the key is 'data') into a pandas DataFrame
df = pd.DataFrame(json_data["data"])

# Select only the desired columns
columns = ["name", "latitude", "longitude", "address", "opens", "closes", "categories", "description", "image"]
df = df[columns]

# Rename columns to match your desired naming convention
df.columns = [
    "Name", "Latitude", "Longitude", "Address",
    "Opens", "Closes", "Categories", "Description", "Image"
]

# Preview the DataFrame
display(df.head())

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
0,*10AM START POINT THE CHRISTMAS SPECTACULAR**,40.754349,-73.986899,"Broadway at 40th Street New York, NY",10:00,11:00,START POINT!,10am Santa is Painting the town Red<br>We will...,https://santacon.nyc/wp-content/uploads/2023/1...
1,The Rutherford,40.751373,-73.993552,"W 33rd St at 8th Ave, New York, NY 10119",10:00,20:00,Huge Venues,Get here early for views from the Roof Deck / ...,https://santacon.nyc/wp-content/uploads/2023/1...
2,Avenida,40.752193,-73.993465,"W. 34 St & 8th Ave, New York, NY 10001",10:00,20:00,Huge Venues,The Holiday Hustle at this rooftop + mexican b...,https://santacon.nyc/wp-content/uploads/2023/1...
3,Taj II,40.741051,-73.992882,"48 W 21st St, New York, NY 10010",12:00,22:00,Huge Venues,Naughty vs Nice at this Sexy Dance Floor Vibin...,https://santacon.nyc/wp-content/uploads/2023/1...
4,Clinton Hall 36,40.750099,-73.984395,"16 W, 36th Street New York, NY 10018",9:00,17:00,Huge Venues,DJs Fun games at this HUGE open spot 🎲🎟️🎁,https://santacon.nyc/wp-content/uploads/2023/1...


In [4]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         75 non-null     object 
 1   Latitude     75 non-null     float64
 2   Longitude    75 non-null     float64
 3   Address      75 non-null     object 
 4   Opens        75 non-null     object 
 5   Closes       75 non-null     object 
 6   Categories   75 non-null     object 
 7   Description  75 non-null     object 
 8   Image        75 non-null     object 
dtypes: float64(2), object(7)
memory usage: 5.4+ KB


None

There are 75 venues including the starting location. We are going to update the columns dtypes to datetime for "Opens" and "Closes". From there we should edit this list down to fewer options as 75 is going to innondate the user with options and make it complicated to choose effective sleigh rides

In [5]:
df['Opens'] = pd.to_datetime(df['Opens'], format='%H:%M').dt.time
df['Closes'] = pd.to_datetime(df['Closes'], format='%H:%M').dt.time

In [6]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         75 non-null     object 
 1   Latitude     75 non-null     float64
 2   Longitude    75 non-null     float64
 3   Address      75 non-null     object 
 4   Opens        75 non-null     object 
 5   Closes       75 non-null     object 
 6   Categories   75 non-null     object 
 7   Description  75 non-null     object 
 8   Image        75 non-null     object 
dtypes: float64(2), object(7)
memory usage: 5.4+ KB


None

In [7]:
display(df.head(3))

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
0,*10AM START POINT THE CHRISTMAS SPECTACULAR**,40.754349,-73.986899,"Broadway at 40th Street New York, NY",10:00:00,11:00:00,START POINT!,10am Santa is Painting the town Red<br>We will...,https://santacon.nyc/wp-content/uploads/2023/1...
1,The Rutherford,40.751373,-73.993552,"W 33rd St at 8th Ave, New York, NY 10119",10:00:00,20:00:00,Huge Venues,Get here early for views from the Roof Deck / ...,https://santacon.nyc/wp-content/uploads/2023/1...
2,Avenida,40.752193,-73.993465,"W. 34 St & 8th Ave, New York, NY 10001",10:00:00,20:00:00,Huge Venues,The Holiday Hustle at this rooftop + mexican b...,https://santacon.nyc/wp-content/uploads/2023/1...


In [8]:
# Create a slice of the starting point for SantaCon
start_loc = df.iloc[0]

In [9]:
display(start_loc)

Name               *10AM START POINT THE CHRISTMAS SPECTACULAR**
Latitude                                               40.754349
Longitude                                             -73.986899
Address                     Broadway at 40th Street New York, NY
Opens                                                   10:00:00
Closes                                                  11:00:00
Categories                                          START POINT!
Description    10am Santa is Painting the town Red<br>We will...
Image          https://santacon.nyc/wp-content/uploads/2023/1...
Name: 0, dtype: object

Our Santas may want the option to go elsewhere, see something nearby. 

In [10]:
bar_filt = df.copy()
bar_filt = bar_filt[bar_filt['Opens']<=time(11, 0)]
bar_filt = bar_filt[bar_filt['Closes']>=time(20, 0)]

In [11]:
display(bar_filt.info())

<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, 1 to 74
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         42 non-null     object 
 1   Latitude     42 non-null     float64
 2   Longitude    42 non-null     float64
 3   Address      42 non-null     object 
 4   Opens        42 non-null     object 
 5   Closes       42 non-null     object 
 6   Categories   42 non-null     object 
 7   Description  42 non-null     object 
 8   Image        42 non-null     object 
dtypes: float64(2), object(7)
memory usage: 3.3+ KB


None

In [12]:
display(bar_filt)

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
1,The Rutherford,40.751373,-73.993552,"W 33rd St at 8th Ave, New York, NY 10119",10:00:00,20:00:00,Huge Venues,Get here early for views from the Roof Deck / ...,https://santacon.nyc/wp-content/uploads/2023/1...
2,Avenida,40.752193,-73.993465,"W. 34 St & 8th Ave, New York, NY 10001",10:00:00,20:00:00,Huge Venues,The Holiday Hustle at this rooftop + mexican b...,https://santacon.nyc/wp-content/uploads/2023/1...
5,5th & Mad,40.749771,-73.982867,"7 E 36th St, New York, NY 10016",11:00:00,20:00:00,Huge Venues,The Snow Ball at this hugh East Side Bar + DJ ...,https://santacon.nyc/wp-content/uploads/2023/1...
8,Solas,40.729445,-73.988059,"232 E 9th St, New York, NY 10003",11:00:00,20:00:00,Huge Venues,Jingle Bell Rockout at this lounge & dance clu...,https://santacon.nyc/wp-content/uploads/2023/1...
10,The Tailor,40.753109,-73.993027,"505 8th Ave, New York, NY 10018",11:00:00,20:00:00,Huge Venues,Mistletoe Mania at this HUGE - 1000+ Elves & S...,https://santacon.nyc/wp-content/uploads/2023/1...
12,Bar 13,40.734531,-73.992180,"121 University Pl, New York, NY 10003",11:00:00,21:00:00,Huge Venues,"Frosty Fest with Drinking & techno, house & hi...",https://santacon.nyc/wp-content/uploads/2023/1...
16,Amsterdam Billiards and Bar,40.731881,-73.989596,"110 E 11th St, New York, NY 10003",11:00:00,20:00:00,East Village Bars,"25 Billiards Tables plus Ping-Pong, Darts, Foo...",https://santacon.nyc/wp-content/uploads/2023/1...
18,Horseshoe Bar,40.725217,-73.981493,"108 Ave B, New York, NY 10009",11:00:00,20:00:00,East Village Bars,No-nonsense Punk & Old School Bar 🎅🧲,https://santacon.nyc/wp-content/uploads/2023/1...
19,Coyote Ugly,40.733034,-73.985653,"233 E. 14th St., New York, NY 10003",11:00:00,21:00:00,East Village Bars,"The One, The Only, Leave Your Bra on the Ceili...",https://santacon.nyc/wp-content/uploads/2023/1...
20,The Laurels,40.732803,-73.984964,"231 2nd Ave, New York, NY 10003",10:00:00,23:00:00,East Village Bars,"Sexy, Cozy Cocktail Bar in East Village 💃🍺⛄",https://santacon.nyc/wp-content/uploads/2023/1...


We have filtered down the potential bars from 74 to 42 options that are open immediately following the initital meet up and late enough that Santas aren't gonna feel rushed. We have come up with 4 Sleigh Ride Themes to choose from:

The Colossal Cheers Circuit (only the largest venues on our list)
Holiday Hors D'oeuvres Hop (a foodie focused bar crawl)
The Midtown Mistletoe March (only midtown locations)
Shamrocks & Stockings Crawl (Irish pub focused)

Let's fliter further and get each Sleigh ride finalized

In [13]:
huge_df = bar_filt[bar_filt['Categories']=='Huge Venues']
display(huge_df)

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
1,The Rutherford,40.751373,-73.993552,"W 33rd St at 8th Ave, New York, NY 10119",10:00:00,20:00:00,Huge Venues,Get here early for views from the Roof Deck / ...,https://santacon.nyc/wp-content/uploads/2023/1...
2,Avenida,40.752193,-73.993465,"W. 34 St & 8th Ave, New York, NY 10001",10:00:00,20:00:00,Huge Venues,The Holiday Hustle at this rooftop + mexican b...,https://santacon.nyc/wp-content/uploads/2023/1...
5,5th & Mad,40.749771,-73.982867,"7 E 36th St, New York, NY 10016",11:00:00,20:00:00,Huge Venues,The Snow Ball at this hugh East Side Bar + DJ ...,https://santacon.nyc/wp-content/uploads/2023/1...
8,Solas,40.729445,-73.988059,"232 E 9th St, New York, NY 10003",11:00:00,20:00:00,Huge Venues,Jingle Bell Rockout at this lounge & dance clu...,https://santacon.nyc/wp-content/uploads/2023/1...
10,The Tailor,40.753109,-73.993027,"505 8th Ave, New York, NY 10018",11:00:00,20:00:00,Huge Venues,Mistletoe Mania at this HUGE - 1000+ Elves & S...,https://santacon.nyc/wp-content/uploads/2023/1...
12,Bar 13,40.734531,-73.992180,"121 University Pl, New York, NY 10003",11:00:00,21:00:00,Huge Venues,"Frosty Fest with Drinking & techno, house & hi...",https://santacon.nyc/wp-content/uploads/2023/1...
74,Circo,40.760420,-73.984250,"1604 Broadway York, NY 10019",11:00:00,22:00:00,Huge Venues,BIGGEST Santacon venue! Get to this new tri-le...,https://santacon.nyc/wp-content/uploads/2024/1...


In [14]:
huge_df = huge_df.drop(5)

In [15]:
display(huge_df)

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
1,The Rutherford,40.751373,-73.993552,"W 33rd St at 8th Ave, New York, NY 10119",10:00:00,20:00:00,Huge Venues,Get here early for views from the Roof Deck / ...,https://santacon.nyc/wp-content/uploads/2023/1...
2,Avenida,40.752193,-73.993465,"W. 34 St & 8th Ave, New York, NY 10001",10:00:00,20:00:00,Huge Venues,The Holiday Hustle at this rooftop + mexican b...,https://santacon.nyc/wp-content/uploads/2023/1...
8,Solas,40.729445,-73.988059,"232 E 9th St, New York, NY 10003",11:00:00,20:00:00,Huge Venues,Jingle Bell Rockout at this lounge & dance clu...,https://santacon.nyc/wp-content/uploads/2023/1...
10,The Tailor,40.753109,-73.993027,"505 8th Ave, New York, NY 10018",11:00:00,20:00:00,Huge Venues,Mistletoe Mania at this HUGE - 1000+ Elves & S...,https://santacon.nyc/wp-content/uploads/2023/1...
12,Bar 13,40.734531,-73.992180,"121 University Pl, New York, NY 10003",11:00:00,21:00:00,Huge Venues,"Frosty Fest with Drinking & techno, house & hi...",https://santacon.nyc/wp-content/uploads/2023/1...
74,Circo,40.760420,-73.984250,"1604 Broadway York, NY 10019",11:00:00,22:00:00,Huge Venues,BIGGEST Santacon venue! Get to this new tri-le...,https://santacon.nyc/wp-content/uploads/2024/1...


In [16]:
midtown_df = bar_filt[bar_filt['Categories'].str.contains('Midtown', case=False, na=False)]

In [17]:
display(midtown_df)

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
26,Backstage Tavern,40.760502,-73.989732,"346 W 46th St, New York, NY 10036",10:00:00,22:00:00,Midtown West Bars,Where Santa's stars align 🌟🎄🩵,https://santacon.nyc/wp-content/uploads/2024/1...
27,Bar Dough,40.760531,-73.989846,"350 W. 46th St, New York, NY 10036",10:00:00,22:00:00,Midtown West Bars,beer & cocktails + wood-fired pizzas 🎅🍕,https://santacon.nyc/wp-content/uploads/2023/1...
28,Blarney Stone,40.750550,-73.994830,"410 8th Ave, New York, NY 10001",11:00:00,20:00:00,Midtown West Bars,Classic NYC Irish Midtown🎅🇮🇪,https://santacon.nyc/wp-content/uploads/2023/1...
34,The Dean,40.754518,-73.989304,"214 W 39th St, New York, NY 10018",11:00:00,22:00:00,Midtown West Bars,Spacious industrial-chic trendy hub 🎅🥃,https://santacon.nyc/wp-content/uploads/2023/1...
39,The Independent,40.754843,-73.987502,"147 W 40th St, New York, NY 10018",11:00:00,20:00:00,Midtown West Bars,Snug and stylish space for notable drinks 🎅🥃,https://santacon.nyc/wp-content/uploads/2023/1...
42,Jack Doyle's,40.752280,-73.992082,"240 W 35th St, New York, NY 10001",10:00:00,21:00:00,Midtown West Bars,DJ at this HUGE Irish Rockin' Haus 🎅🍀,https://santacon.nyc/wp-content/uploads/2023/1...
43,John Sullivan's,40.751815,-73.990753,"210 W 35th St, New York, NY 10001",10:00:00,21:00:00,Midtown West Bars,2 Levels & Amazing Drink Specials 👯🕶️,https://santacon.nyc/wp-content/uploads/2023/1...
45,The Liberty,40.749799,-73.985657,"29 W 35th St, New York, NY 10001",10:00:00,20:00:00,Midtown West Bars,Amazing fusion between classic & contemporary ...,https://santacon.nyc/wp-content/uploads/2023/1...
46,Printers Alley,40.755506,-73.988588,"215 West 40th St, New York, NY 10018",10:00:00,20:00:00,Midtown West Bars,4 Floor Sprawling Irish Pub 🎅🏠,https://santacon.nyc/wp-content/uploads/2023/1...
49,Peter Dillons 36th,40.749603,-73.983317,"2 E 36th St, New York, NY 10016",10:00:00,20:00:00,Midtown East Bars,Beers & Cocktails for Santa 🎅🇮🇪,https://santacon.nyc/wp-content/uploads/2023/1...


In [18]:
midtown_exclude_list = ["Bar Dough", "Blarney Stone", "Jack Doyle\'s",
                        "Printers Alley", "Peter Dillons 36th", "Peter Dillons Pub 40th",
                        "Playwright Irish Pub", "Slattery's", "Westbury", 
                        "Walters Cottage", "Celtic Rail", "Jameson's",
                        "The Dean", "The Independent"]
midtown_df = midtown_df[~midtown_df['Name'].isin(midtown_exclude_list)]

In [19]:
display(midtown_df)

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
26,Backstage Tavern,40.760502,-73.989732,"346 W 46th St, New York, NY 10036",10:00:00,22:00:00,Midtown West Bars,Where Santa's stars align 🌟🎄🩵,https://santacon.nyc/wp-content/uploads/2024/1...
43,John Sullivan's,40.751815,-73.990753,"210 W 35th St, New York, NY 10001",10:00:00,21:00:00,Midtown West Bars,2 Levels & Amazing Drink Specials 👯🕶️,https://santacon.nyc/wp-content/uploads/2023/1...
45,The Liberty,40.749799,-73.985657,"29 W 35th St, New York, NY 10001",10:00:00,20:00:00,Midtown West Bars,Amazing fusion between classic & contemporary ...,https://santacon.nyc/wp-content/uploads/2023/1...
62,La Macarena NYC,40.760747,-73.986387,"234 W 48th St, New York, NY 10036",11:00:00,21:00:00,Midtown West Bars,"Santa loves Latin Food, Hooka & Party Bar in T...",https://santacon.nyc/wp-content/uploads/2024/1...
66,Karaoke City,40.747540,-73.986076,"22 W 32nd St 7th Floor, New York, NY 10001",11:00:00,21:00:00,Midtown East Bars,All I want for Christmas is on the 7th Floor f...,https://santacon.nyc/wp-content/uploads/2024/1...
67,Pennsylvania 6,40.748984,-73.989846,"132 W 31st St, New York, NY 10001",10:00:00,20:00:00,Midtown West Bars,Sleigh at this huge Midtown masterpiece 🛷🍸🎄,https://santacon.nyc/wp-content/uploads/2024/1...


In [20]:
food_word_list = ['food', 'pizza']
food_pattern = '|'.join(food_word_list)
# Filter for Grub or descriptions that mention food
food_bars = bar_filt[
    (bar_filt['Categories']== "Grub") |
    (bar_filt['Description'].str.contains(food_pattern,
                                          case=False, na=False))
]

# Preview filtered DataFrame
display(food_bars)

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
16,Amsterdam Billiards and Bar,40.731881,-73.989596,"110 E 11th St, New York, NY 10003",11:00:00,20:00:00,East Village Bars,"25 Billiards Tables plus Ping-Pong, Darts, Foo...",https://santacon.nyc/wp-content/uploads/2023/1...
27,Bar Dough,40.760531,-73.989846,"350 W. 46th St, New York, NY 10036",10:00:00,22:00:00,Midtown West Bars,beer & cocktails + wood-fired pizzas 🎅🍕,https://santacon.nyc/wp-content/uploads/2023/1...
31,Brooklyn Deli Times Square,40.757228,-73.986979,"211 West 43rd St, New York, NY 10036",10:00:00,20:00:00,Grub,Satin Dolls performing Christmas classics & th...,https://santacon.nyc/wp-content/uploads/2024/1...
47,Montagu's Gusto,40.745833,-73.975552,"645 2nd Ave, New York, NY 10016",09:00:00,23:00:00,Grub,Artisan eatery bringing new flavors to Santaco...,https://santacon.nyc/wp-content/uploads/2024/1...
62,La Macarena NYC,40.760747,-73.986387,"234 W 48th St, New York, NY 10036",11:00:00,21:00:00,Midtown West Bars,"Santa loves Latin Food, Hooka & Party Bar in T...",https://santacon.nyc/wp-content/uploads/2024/1...
65,Kinky's Dessert Bar,40.722162,-73.988571,"181 Orchard St, New York, NY 10002",11:00:00,23:00:00,Grub,A Dessert Shop with Booze...Let's Get Kinky! 👄,https://santacon.nyc/wp-content/uploads/2023/1...
70,Cafe Flor,40.744262,-73.999222,"218 8th Ave, New York, NY 10011",10:00:00,20:00:00,Grub,Santa's cozy Coffee Shop / Bar & quick bites 🥖🍸🎄🎅,https://lh3.googleusercontent.com/p/AF1QipO49m...


In [21]:
food_bars = food_bars.drop(31)

In [22]:
display(food_bars)

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
16,Amsterdam Billiards and Bar,40.731881,-73.989596,"110 E 11th St, New York, NY 10003",11:00:00,20:00:00,East Village Bars,"25 Billiards Tables plus Ping-Pong, Darts, Foo...",https://santacon.nyc/wp-content/uploads/2023/1...
27,Bar Dough,40.760531,-73.989846,"350 W. 46th St, New York, NY 10036",10:00:00,22:00:00,Midtown West Bars,beer & cocktails + wood-fired pizzas 🎅🍕,https://santacon.nyc/wp-content/uploads/2023/1...
47,Montagu's Gusto,40.745833,-73.975552,"645 2nd Ave, New York, NY 10016",09:00:00,23:00:00,Grub,Artisan eatery bringing new flavors to Santaco...,https://santacon.nyc/wp-content/uploads/2024/1...
62,La Macarena NYC,40.760747,-73.986387,"234 W 48th St, New York, NY 10036",11:00:00,21:00:00,Midtown West Bars,"Santa loves Latin Food, Hooka & Party Bar in T...",https://santacon.nyc/wp-content/uploads/2024/1...
65,Kinky's Dessert Bar,40.722162,-73.988571,"181 Orchard St, New York, NY 10002",11:00:00,23:00:00,Grub,A Dessert Shop with Booze...Let's Get Kinky! 👄,https://santacon.nyc/wp-content/uploads/2023/1...
70,Cafe Flor,40.744262,-73.999222,"218 8th Ave, New York, NY 10011",10:00:00,20:00:00,Grub,Santa's cozy Coffee Shop / Bar & quick bites 🥖🍸🎄🎅,https://lh3.googleusercontent.com/p/AF1QipO49m...


In [23]:
irish_bars = bar_filt[bar_filt['Description'].str.contains('Irish|🇮🇪', case=False, na=False)]
display(irish_bars)

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
28,Blarney Stone,40.750550,-73.994830,"410 8th Ave, New York, NY 10001",11:00:00,20:00:00,Midtown West Bars,Classic NYC Irish Midtown🎅🇮🇪,https://santacon.nyc/wp-content/uploads/2023/1...
42,Jack Doyle's,40.752280,-73.992082,"240 W 35th St, New York, NY 10001",10:00:00,21:00:00,Midtown West Bars,DJ at this HUGE Irish Rockin' Haus 🎅🍀,https://santacon.nyc/wp-content/uploads/2023/1...
46,Printers Alley,40.755506,-73.988588,"215 West 40th St, New York, NY 10018",10:00:00,20:00:00,Midtown West Bars,4 Floor Sprawling Irish Pub 🎅🏠,https://santacon.nyc/wp-content/uploads/2023/1...
49,Peter Dillons 36th,40.749603,-73.983317,"2 E 36th St, New York, NY 10016",10:00:00,20:00:00,Midtown East Bars,Beers & Cocktails for Santa 🎅🇮🇪,https://santacon.nyc/wp-content/uploads/2023/1...
50,Peter Dillons Pub 40th,40.750030,-73.976846,"130 E 40th St, New York, NY 10016",10:00:00,20:00:00,Midtown East Bars,Beers & Cocktails for Santa 🎅🇮🇪,https://santacon.nyc/wp-content/uploads/2023/1...
52,Playwright Irish Pub,40.749949,-73.985427,"27 W 35th St, New York, NY 10001",10:00:00,21:00:00,Midtown West Bars,Santa's Winter Warmer Bar 🎅🇮🇪,https://santacon.nyc/wp-content/uploads/2023/1...
54,Slattery's,40.749420,-73.983008,"8 E 36th St, New York, NY 10016",09:00:00,20:00:00,Midtown East Bars,Irish pub with Guinness on Draft 🎅💃,https://santacon.nyc/wp-content/uploads/2023/1...
56,Westbury,40.751425,-73.983823,"20 W 38th St, New York, NY 10018",11:00:00,21:00:00,Midtown West Bars,Great Irish Pubs NYC Best Sports Bar Midtown M...,https://santacon.nyc/wp-content/uploads/2024/1...
57,Walters Bar,40.749495,-73.995850,"389 8th Ave, New York, NY 10001",11:00:00,20:00:00,Chelsea,"beers, darts & a pool table 🎅🇮🇪",https://santacon.nyc/wp-content/uploads/2023/1...
61,Celtic Rail,40.750069,-73.989906,"137 W 33rd St, New York, NY 10120",09:00:00,23:59:00,Midtown East Bars,Madison Square Garden's staple Irish Locals Ha...,https://santacon.nyc/wp-content/uploads/2023/1...


In [24]:
irish_exclusion_list = ['Blarney Stone', 'Peter Dillons 36th', 
                        'Peter Dillons Pub 40th', "Slattery\'s",
                        'Westbury']
irish_bars = irish_bars[~irish_bars['Name'].isin(irish_exclusion_list)]

In [25]:
display(irish_bars)

,Name,Latitude,Longitude,Address,Opens,Closes,Categories,Description,Image
42,Jack Doyle's,40.752280,-73.992082,"240 W 35th St, New York, NY 10001",10:00:00,21:00:00,Midtown West Bars,DJ at this HUGE Irish Rockin' Haus 🎅🍀,https://santacon.nyc/wp-content/uploads/2023/1...
46,Printers Alley,40.755506,-73.988588,"215 West 40th St, New York, NY 10018",10:00:00,20:00:00,Midtown West Bars,4 Floor Sprawling Irish Pub 🎅🏠,https://santacon.nyc/wp-content/uploads/2023/1...
52,Playwright Irish Pub,40.749949,-73.985427,"27 W 35th St, New York, NY 10001",10:00:00,21:00:00,Midtown West Bars,Santa's Winter Warmer Bar 🎅🇮🇪,https://santacon.nyc/wp-content/uploads/2023/1...
57,Walters Bar,40.749495,-73.995850,"389 8th Ave, New York, NY 10001",11:00:00,20:00:00,Chelsea,"beers, darts & a pool table 🎅🇮🇪",https://santacon.nyc/wp-content/uploads/2023/1...
61,Celtic Rail,40.750069,-73.989906,"137 W 33rd St, New York, NY 10120",09:00:00,23:59:00,Midtown East Bars,Madison Square Garden's staple Irish Locals Ha...,https://santacon.nyc/wp-content/uploads/2023/1...
63,Jameson's,40.754795,-73.968974,"920 2nd Ave, New York, NY 10017",11:00:00,23:00:00,Midtown East Bars,Real NYC Bar hang for Santa 🎅🇮🇪,https://santacon.nyc/wp-content/uploads/2024/1...
71,McKenna's,40.739637,-74.002151,"250 W 14th St, New York, NY 10011",11:00:00,20:00:00,Chelsea,Santa's Time-tested pub + bar bites 🎅🇮🇪,https://lh3.googleusercontent.com/p/AF1QipN3xQ...


Okay--the Sleigh ride venues have been finalized. Each ride has access to 6-7 venues. We will now get the name, coordinates and address for the 3 hotels doing deals. 

## Hotels

In [26]:
with open("hotels.json", "r") as file2:
    json_hotel = json.load(file2)

In [27]:
# Convert the nested JSON object (assuming the key is 'Hotel') into a pandas DataFrame
hotel_df = pd.DataFrame(json_hotel["Hotels"])

# Select only the desired columns
columns = ["name", 'short', "latitude", "longitude", "address", "description", "image"]
hotel_df = hotel_df[columns]

# Rename columns to match your desired naming convention
hotel_df.columns = [
    "Name", "Short", "Latitude", "Longitude", "Address",
    "Description", "Image"
]

# Preview the DataFrame
display(hotel_df.head())

,Name,Short,Latitude,Longitude,Address,Description,Image
0,Arthouse Hotel NYC,Arthouse,40.782253,-73.980271,"2178 Broadway, New York, NY, 10024","Arthouse Hotel New York City has brought hip, ...",https://image-tc.galaxy.tf/wijpeg-1nbmvmxtb567...
1,Hotel Indigo LES,Indigo,40.721923,-73.987749,"171 Ludlow St, New York, NY 10002",Whether you’re snapping photos of our spectacu...,https://digital.ihg.com/is/image/ihg/hotel-ind...
2,The High Line Hotel,Highline,40.746105,-74.004977,"180 10th Ave, New York, NY 10011",Nestled in the heart of Chelsea’s buzzing gall...,https://thehighlinehotel.com/wp-content/upload...
3,Virgin Hotels NYC,Virgin,40.746757,-73.988796,"1227 Broadway, New York, NY 10001",Discover the best bed in all five boroughs in ...,https://newyorkyimby.com/wp-content/uploads/20...


We now have every component we need for assessing optimal routes. Let's devise an algorithm to find optimal routes and compare it to a baseline random route. 

In [28]:
def coordinates(row):
    return (row['Longitude'], row['Latitude'])

In [29]:
def create_distance_matrix (start, sleigh_ride, hotel):
    coordinate_list = [coordinates(start)]
    for ride in sleigh_ride:
        location.append(coordinates(ride))
    location.append(coordinates(hotel))
    
    matrix = client.distance_matrix(
        locations = coordinate_list,
        profile = 'driving-car',
        metrics = ['distance']
    )
    return matrix['distances']

In [30]:
def find_optimal_route(start_idx, end_idx, distance_matrix):
    num_locations = len(distance_matrix)
    all_points = list(range(num_locations))

    # Intermediate points (excluding start and end)
    intermediate_points = [i for i in all_points if i != start_idx and i != end_idx]

    min_distance = float('inf')
    best_route = None

    # Generate all possible permutations of intermediate points
    for perm in permutations(intermediate_points):
        # Complete the route: start -> permuted points -> end
        route = [start_idx] + list(perm) + [end_idx]

        # Calculate the total distance for this route
        total_distance = sum(
            distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1)
        )

        # Update minimum distance and best route if necessary
        if total_distance < min_distance:
            min_distance = total_distance
            best_route = route

    return best_route, min_distance

In [31]:
# Initialize the client with API key
client = openrouteservice.Client(key="5b3ce3597851110001cf6248ae3b610c01274cf597df55dd505dd419")


In [32]:
# start_idx = 0  # Index of start location
# end_idx = len(locations) - 1  # Index of end location

# optimal_route, minimal_distance = find_optimal_route(start_idx, end_idx, distance_matrix)

#  Make a loop for the displaying code below--
# # Step 6: Display the result
# print("Optimal Route (indices):", optimal_route)
# print("Minimal Total Distance (meters):", minimal_distance)

# # Map indices to actual locations
# optimal_route_locations = [locations[i] for i in optimal_route]
# print("Optimal Route (coordinates):", optimal_route_locations)

In [33]:
def make_route_base (start, route_df, hotels):
    start = start.to_frame().T
    # Concatenate the start location, route DataFrame, and a single hotel row for each route
    art_route = pd.concat([start, route_df, hotels.iloc[[0]].copy()], ignore_index=True).reset_index(drop=True)
    ind_route = pd.concat([start, route_df, hotels.iloc[[1]].copy()], ignore_index=True).reset_index(drop=True)
    high_route = pd.concat([start, route_df, hotels.iloc[[2]].copy()], ignore_index=True).reset_index(drop=True)
    vir_route = pd.concat([start, route_df, hotels.iloc[[3]].copy()], ignore_index=True).reset_index(drop=True)
    
    return art_route, ind_route, high_route, vir_route

In [34]:
start_loc_cleaned = start_loc.drop(['Opens', 'Closes', 'Categories'])
huge_bars_cleaned = huge_df.drop(['Opens', 'Closes', 'Categories'], axis=1)
midtown_bars_cleaned = midtown_df.drop(['Opens', 'Closes', 'Categories'], axis=1)
food_bars_cleaned = food_bars.drop(['Opens', 'Closes', 'Categories'], axis=1)
irish_bars_cleaned = irish_bars.drop(['Opens', 'Closes', 'Categories'], axis=1)

In [35]:
# Huge venue routes
huge_arthouse, huge_indigo, huge_highline, huge_virgin = \
    make_route_base(start_loc_cleaned, huge_bars_cleaned, hotel_df)

# Midtown venue routes
midtown_arthouse, midtown_indigo, midtown_highline, midtown_virgin = \
    make_route_base(start_loc_cleaned, midtown_bars_cleaned, hotel_df)

# Food venue routes
food_arthouse, food_indigo, food_highline, food_virgin = \
    make_route_base(start_loc_cleaned, food_bars_cleaned, hotel_df)

# Irish venue routes
irish_arthouse, irish_indigo, irish_highline, irish_virgin = \
    make_route_base(start_loc_cleaned, irish_bars_cleaned, hotel_df)

In [36]:
# Optimal huge routes
order_art = [0, 1, 5, 3, 2, 4, 6, 7]
order_ind = [0, 6, 4, 2, 1, 5, 3, 7]
order_high = [0, 2, 3, 6, 4, 1, 5, 7]
order_vir = [0, 2, 1, 6, 3, 4, 5, 7]

# optimized ordered DataFrames
huge_arthouse_opt = huge_arthouse.iloc[order_art]
huge_indigo_opt = huge_indigo.iloc[order_ind]
huge_highline_opt = huge_highline.iloc[order_high]
huge_virgin_opt = huge_virgin.iloc[order_vir]

In [37]:
# Optimal midtown routes
order_art = [0, 5, 6, 3, 2, 4, 1, 7]
order_ind = [0, 4, 1, 3, 2, 5, 6, 7]
order_high = [0, 4, 1, 3, 2, 5, 6, 7]
order_vir = [0, 4, 1, 3, 2, 5, 6, 7]

# optimized ordered DataFrames
midtown_arthouse_opt = midtown_arthouse.iloc[order_art].reset_index(drop=True)
midtown_indigo_opt = midtown_indigo.iloc[order_ind].reset_index(drop=True)
midtown_highline_opt = midtown_highline.iloc[order_high].reset_index(drop=True)
midtown_virgin_opt = midtown_virgin.iloc[order_vir].reset_index(drop=True)

In [38]:
# Optimal food routes
order_art = [0, 3, 5, 1, 6, 4, 2, 7]
order_ind = [0, 2, 4, 6, 3, 1, 5, 7]
order_high = [0, 4, 6, 1, 5, 2, 3, 7]
order_vir = [0, 1, 2, 6, 5, 3, 4, 7]

# optimized ordered DataFrames
food_arthouse_opt = food_arthouse.iloc[order_art].reset_index(drop=True)
food_indigo_opt = food_indigo.iloc[order_ind].reset_index(drop=True)
food_highline_opt = food_highline.iloc[order_high].reset_index(drop=True)
food_virgin_opt = food_virgin.iloc[order_vir].reset_index(drop=True)

In [39]:
# Optimal irish routes
order_art = [0, 1, 3, 5, 7, 4, 2, 6, 8]
order_ind = [0, 1, 3, 5, 7, 4, 2, 6, 8]
order_high = [0, 1, 2, 6, 3, 5, 4, 7, 8]
order_vir = [0, 1, 2, 6, 3, 5, 7, 4, 8]

# optimized ordered DataFrames
irish_arthouse_opt = irish_arthouse.iloc[order_art].reset_index(drop=True)
irish_indigo_opt = irish_indigo.iloc[order_ind].reset_index(drop=True)
irish_highline_opt = irish_highline.iloc[order_high].reset_index(drop=True)
irish_virgin_opt = irish_virgin.iloc[order_vir].reset_index(drop=True)